In [1]:
from env import SCIENTIST

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time

In [3]:
import importlib # for reloading .py files
# WARNING: updates of config may require restart of kernel if reload is unsuccessful

import config
# reload config without restarting the kernel
importlib.reload(config)
from config import actions, no_sequences, sequence_length, number_of_classes, DATA_PATH, WEIGHTS_PATH, facemesh_included, is_martin

import utils
# reload utils without restarting the kernel
importlib.reload(utils)
from utils import mediapipe_detection, extract_keypoints, draw_styled_landmarks, prob_viz


import model
# reload model without restarting the kernel
importlib.reload(model)
from model import create_model, build_which_model

# CHOOSE MODEL / WEIGHTS

In [4]:
# name of the weights file (without ".keras" ending) 
# also used as model name - dot not allowed, thus deleted, and file renamed
model_name = "weights_wo_facemesh_LSTM_tanh_softmax_NM20_testsize0.3"
weights_name = model_name

# BUILD THE MODEL - CHRIS

In [5]:
# OPTIONAL model() arguments
# available model_types: "LSTM", "Conv2D"
# tested activation functions: "tanh", "relu"; OTHERS: see below 
# tested activations for last neural layer: "softmax", "sigmoid"; OTHERS: "linear", "tanh", "relu", "softplus", "softsign", "selu", "elu", "exponential"
# neural multiplicator: multiply neurons per layer by this amount (default = 1); the higher NM, the longer the computation
# output will change according if facemesh_included = True or False

# reLU = Rectified Linear Activation Function: returns 0 if it receives any negative input, but for any positive value x it returns that value back.  It helps the model to account for non-linearity.
# Sigmoid: Outputs a value between 0 and 1, making it suitable for binary classification or multilabel classification tasks where each label is predicted independently.
model_type, activation_function, activation, neural_multiplicator = "LSTM", "tanh", "softmax", 20
model = create_model(model_type, activation_function, activation, neural_multiplicator)

['categorical_accuracy', 'accuracy', 'Precision', 'Recall']


# CALL THE MODEL - Martin

In [6]:
if is_martin: 
    model = build_which_model(model_name)

## Loading weights

In [7]:
model.load_weights(os.path.join(WEIGHTS_PATH, weights_name + ".keras"))

# PROBABILITY VIZUALIZATION - REAL TIME TESTING

In [8]:
import mediapipe as mp
from scipy.stats import mode 

In [9]:
from utils import mp_holistic, mp_drawing
import datetime

In [11]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.4 # 40% 
frame_counter = 0
last_word = ""
num_of_words = 5

length_of_sequence = 30
no_of_reps_to_accept = 5
silent_sign_name = "NoHands"

# picture of a static person using no hands as fall-back option for model
static_frame = np.load(os.path.join(DATA_PATH, "_.npy"))

cap = cv2.VideoCapture(0) # I have changed the 1 to 0 in order to open successfully the camera

#TODO: put recording in subfolder
# recording
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
video_file_name = f"recording_{timestamp}.avi"
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(video_file_name, fourcc, 10.0, (int(cap.get(3)), int(cap.get(4))))
# /recording


# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        frame_counter += 1

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        # dynamically adjust the length of the sequence using the variable from above
        sequence = sequence[-length_of_sequence:]

        if len(sequence) == length_of_sequence:

            zero_frame = np.zeros_like(sequence[0])  # Creates a zero array with the same shape as a frame

            # overwrites zero_frame with the static
            zero_frame = static_frame

            padding = [zero_frame] * (int((30 - len(sequence))/2))

            # pad the actual sequence from left and right
            padded_sequence = padding + sequence + padding

            # Ensure that the padded sequence is exactly 30 frames long
            padded_sequence = padded_sequence[:30]

            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)], round(max(res),2))
            predictions.append(np.argmax(res))
            #  my_predictions = list(dict.fromke<ys(predictions)) --> another way how to get rid of duplicates and keep the order of the elements in the list


        #3. Viz logic
            # if np.unique(predictions[-2:])[0]==np.argmax(res): <-- wrong line of code since .unique sorts the elements in the list alphabetically!!!
            
            # if mode(predictions[-3:])[0]==np.argmax(res):


            is_silent_sign = actions[np.argmax(res)] == silent_sign_name

            if (np.unique(predictions[-no_of_reps_to_accept:])[0]==np.argmax(res) 
                and (np.unique(predictions[-no_of_reps_to_accept:]).shape[0]==1) # additional condition to check if the returned unique list has only one element
                and not is_silent_sign
                ): 


                if (res[np.argmax(res)] > threshold) and (actions[np.argmax(res)] != last_word): 
                    if len(sentence) >= num_of_words:
                        time.sleep(3) 
                        sentence = []
                        sentence.append(actions[np.argmax(res)])
                        last_word = actions[np.argmax(res)]
                        frame_counter = 0
                    elif len(sentence) > 0: 
                        sentence.append(actions[np.argmax(res)])
                        last_word = actions[np.argmax(res)]
                        frame_counter = 0
                    elif len(sentence) == 0:
                        sentence.append(actions[np.argmax(res)])
                        last_word = actions[np.argmax(res)]
                        frame_counter = 0
                    # the same thing:
                    # else:
                    #     sentence.append(actions[np.argmax(res)])
                        
                        
            # if len(sentence) > 5:
            #     sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, threshold)
       
        if frame_counter == 15:
            sentence = []
            last_word = ""
            frame_counter = 0   

        cv2.putText(image, f"{frame_counter}, last word {last_word}", (50,50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.rectangle(image, (0, 430), (640, 480), (128, 128, 128), -1) # TODO : make it more transperant
        cv2.putText(image, ', '.join(sentence), (50,465),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # write the frame in the video recording
        out.write(image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('t'):
            break
    
    #release the recording
    out.release()

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti